# ゲート付きRNN

RNN層にゲートと呼ばれる機構を追加し、より長期的な文脈の情報を保持できるようにしたもの。

In [1]:
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(width=20, with_test=True, label="ppl train", round=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 学習データの用意

In [3]:
n_data = 20000
textfile = f'data/jawiki_{n_data}.txt'
tokenizer_prefix = f'models/tokenizer_jawiki_{n_data}'

In [4]:
with open(textfile) as f:
    data = f.read().splitlines()

In [5]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model')
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()
pad_id = sp.pad_id()

data_ids = sp.encode(data)
for ids in data_ids:
    ids.insert(0, bos_id)
    ids.append(eos_id)

print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[1, 12, 20, 528, 495, 283, 48, 558, 52, 3542]

In [6]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self._n_samples = len(data_ids)
        self.data = [torch.tensor(ids) for ids in data_ids]

    def __getitem__(self, idx):
        in_text = self.data[idx][:-1]
        out_text = self.data[idx][1:]
        return in_text, out_text

    def __len__(self):
        return self._n_samples

def collate_fn(batch):
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 32
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

sample = next(iter(train_loader))
sample[0].shape

torch.Size([32, 1247])


---

## ゲート

あるデータをどれくらい通すかを示したもの。具体的には0~1の要素を持つ対象のデータと同じサイズのベクトル。

例えば、$z=(0.2,0.5,0.9)$というゲートは一つ目の要素を2割、二つ目の要素を5割、三つ目の要素を9割通すゲートである。

In [7]:
z = torch.tensor([0.2, 0.5, 0.9])

このゲートに適当なデータとして3次元ベクトル$x=(1,2,3)$を掛けてみる。

In [8]:
x = torch.tensor([1, 2, 3])
y = x * z
y

tensor([0.2000, 1.0000, 2.7000])

これで、元の入力の**一部を通した**ことになった。これがゲートの役目。

このゲートをRNN層に取り入れる。以下のように扱う。

$$
\begin{align}
\tilde h &= h \odot z \\
z &= \text{gate}(h) = \sigma(\text{fc}(h))
\end{align}
$$

あるデータ$h$に対し、適当な関数$\text{gate}$を用いてゲート$z$を作り、$h$と$z$の要素ごとの積をとる。ゲートを作る関数は基本的に全結合層$\text{fc}$とシグモイド関数$\sigma$を組み合わせて作る。もちろんこの全結合層は学習可能。

ゲート部分をPyTorchで実装すると以下。

In [9]:
class Gate(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [10]:
input_size = 3
gate = Gate(input_size)

x = torch.randn(input_size) # 適当なデータ
gate_x = gate(x)
print("data:", x)
print("gate:", gate_x)

y = x * gate_x
print("output:", y)

data: tensor([-0.3042,  0.0647, -1.0536])
gate: tensor([0.7080, 0.5506, 0.5406], grad_fn=<SigmoidBackward0>)
output: tensor([-0.2154,  0.0356, -0.5696], grad_fn=<MulBackward0>)



---

## GRU

*Gated Recurrent Unit*

ゲート付きRNNの一種。

一旦RNNの復習をしよう。


RNNはある時間$t$の入力$x_t$に対して以下のような演算で出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

この$x_t$と$h_{t-1}$の全結合の部分をまとめて、一つの全結合$\mathrm{fc}(x,h)$で表すことにしよう。

$$
\begin{align}
h_t &= \mathrm{tanh}(\mathrm{fc}(x_t,h_{t-1})) \\
\mathrm{fc}(x,h) &= W_x x + b_x + W_h h + b_h
\end{align}
$$

んで、$\mathrm{fc}(x,h)$の実装もしておこう。

In [11]:
class FullyConnected(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, hidden_size)
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h):
        return self.fc_input(x) + self.fc_hidden(h)

では、GRUの構造を見ていこう。GRUは以下のような演算で出力値$h_t$を決定する。

$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,h_{t-1})) \\
z_t &= \text{gate}(x_t,h_{t-1}) = \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
\end{align}
$$

$\sigma(x)$はsigmoid関数。なお、このGRUは一般的なものより少し簡略化されているので注意。

RNNでは新たなデータ$\tilde h_t$がそのまま出力されていたが、GRUでは、新たなデータ$\tilde h_t$を古いデータ$h_{t-1}$に足して出力する。そして、その際の比率をゲート$z_t$で決める。$z_t$は$h_{t-1}$をどれだけ通すかを表すもので、$x_t$と$h_{t-1}$によって決まる。

このように、GRUではゲートを用いて新たなデータをどれだけ取り入れるべきか、そして古いデータをどれだけ捨てるか考えることが出来る。この枠組みの下で学習を行うことで、長期的に保持すべきデータをしっかりと保持できるようになることが期待される。

単一時間のGRUを実装してみよう。まずGRU内部で用いるゲートをつくる。

In [24]:
class Gate(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc = FullyConnected(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, h):
        return self.sigmoid(self.fc(x, h))

$x$と$h$の二つを入力するので`nn.Sequential`は使えない。

でこれを使ってGRUを実装するとこう。

In [25]:
class SimpleGRUCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc = FullyConnected(input_size, hidden_size)
        self.gate = Gate(input_size, hidden_size)

    def forward(self, x, h):
        h_new = F.tanh(self.fc(x, h))
        z = self.gate(x, h)
        h = (1 - z)*h_new + z*h
        return h

In [26]:
batch_size = 32
embed_size = 128
hidden_size = 256
x = torch.randn(batch_size, embed_size)
h = torch.randn(batch_size, hidden_size)

gru = SimpleGRUCell(embed_size, hidden_size)
h_new = gru(x, h)
h_new.shape

torch.Size([32, 256])

上記のモデルは通常のGRUを私が簡略化したもの。通常のGRUは、上記のモデルにゲートを一つ追加した以下のモデルである。

$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,r_t \odot h_{t-1})) \\
z_t &= \text{gate}_z(x_t,h_{t-1}) = \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
r_t &= \text{gate}_r(x_t,h_{t-1}) = \sigma(\mathrm{fc}_{r}(x_t,h_{t-1})) \\
\end{align}
$$

新なデータ$\tilde h_t$を生成する際に、古いデータ$h_{t-1}$をどれだけ考慮するかを決めるゲート$r_t$が追加されている。

In [29]:
class GRUCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_input = FullyConnected(input_size, hidden_size)
        self.gate_update = Gate(input_size, hidden_size)
        self.gate_reset = Gate(input_size, hidden_size)

    def forward(self, x, h):
        r = self.gate_reset(x, h)
        h_new = F.tanh(self.fc_input(x, r * h))
        z = self.gate_update(x, h)
        h = (1 - z)*h_new + z*h
        return h

RNN同様、PyTorchにクラスが用意されている。

- `GRUCell`: https://pytorch.org/docs/stable/generated/torch.nn.GRUCell.html
- `GRU`: https://pytorch.org/docs/stable/generated/torch.nn.GRU.html

In [14]:
gru = nn.GRU(input_size, input_size)


---

## LSTM

*Long Short-Term Memory*

長短期記憶

GRUの進化版。考え方はGRUと同じで、RNNにゲートを取り入れてイイ感じにしたもの。ちなみに、GRUよりLSTMの方が先に提案されている。GRUはLSTMの簡易版として後から提案された。

LSTMには出力する隠れ状態$h_t$だけでなく、**記憶セル**と呼ばれる変数$c_t$を持つ。記憶セルはLSTMの外に出力されることはなく、LSTM内部でのみ使用される。

まず簡単に文字で説明する。記憶セル$c_t$がGRUでの隠れ状態$h_t$に当たり、ゲートを用いた不要な情報の削除と新たな情報の追加が行われる。なおゲートの生成には入力$x_t$と前の隠れ状態$h_{t-1}$を用いる（記憶セルは用いない）。そしてこの記憶セルを活性化関数に通したものをLSTMの出力=隠れ状態$h_t$とする。

数式で見てみよう。

$$
\begin{align}
h_t &= o_t \odot \mathrm{tanh}(c_t) \\
c_t &= f_t \odot c_{t-1} + i_t \odot \tilde c_t \\
\tilde c_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde c}(x_t,h_{t-1})) \\
i_t &= \sigma(\mathrm{fc}_{i}(x_t,h_{t-1})) \\
f_t &= \sigma(\mathrm{fc}_{f}(x_t,h_{t-1})) \\
o_t &= \sigma(\mathrm{fc}_{o}(x_t,h_{t-1})) \\
\end{align}
$$


- $\tilde c_t$: 新たな情報。
- $i_t$: inputゲート。新たな情報$\tilde c_t$をどれだけ取り入れるかを決める。
- $f_t$: forgetゲート。古い情報$c_{h-1}$をどれだけ保持するかを決めるゲート。
- $o_t$: outputゲート。出力する隠れ状態の量を決めるゲート。

GRUでは1つのゲートを用いて新たな情報と古い情報の比率を決めていたが、LSTMでは別々のゲートを用いて決める。

実装は以下の通り。

In [31]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.gate_input = Gate(input_size, hidden_size)
        self.gate_forget = Gate(input_size, hidden_size)
        self.gate_output = Gate(input_size, hidden_size)
        self.fc = FullyConnected(input_size, hidden_size)

    def forward(self, x, hc):
        h, c = hc
        c_new = F.tanh(self.fc(x, h))
        i = self.gate_input(x, h)
        f = self.gate_forget(x, h)
        o = self.gate_output(x, h)
        c = f*c + i*c_new
        h = o * F.tanh(c)
        return h, c

全ての時間を一括で処理する`LSTM`も実装しておこう。

In [32]:
class LSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.lstm_cell = LSTMCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x, hc=None):
        """
            x: (batch_size, seq_len, input_size)
            h: (batch_size, hidden_size)
        """
        if hc is None:
            h = torch.zeros(x.size(0), self.hidden_size).to(x.device)
            c = torch.zeros(x.size(0), self.hidden_size).to(x.device)
        else:
            h, c = hc
        hs = []
        x = x.transpose(0, 1) # (seq_len, batch_size, input_size)
        for xi in x:
            (h, c) = self.lstm_cell(xi, (h, c))
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        hs = hs.transpose(0, 1) # (batch_size, seq_len, hidden_size)
        return hs, (h, c)

In [34]:
batch_size = 32
seq_len = 10
embed_size = 128
hidden_size = 256

x = torch.randn(batch_size, seq_len, embed_size)
h = torch.randn(batch_size, hidden_size)
c = torch.randn(batch_size, hidden_size)

lstm = LSTM(embed_size, hidden_size)
hs, (h, c) = lstm(x, (h, c))
hs.shape, h.shape, c.shape

(torch.Size([32, 10, 256]), torch.Size([32, 256]), torch.Size([32, 256]))

こちらもPyTorchにクラスが用意されている。

- `LSTMCell`: https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html
- `LSTM`: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

In [38]:
x = torch.randn(batch_size, seq_len, embed_size)
h = torch.randn(1, batch_size, hidden_size)
c = torch.randn(1, batch_size, hidden_size)

lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
hs, (h, c) = lstm(x, (h, c))
hs.shape, h.shape, c.shape

(torch.Size([32, 10, 256]), torch.Size([1, 32, 256]), torch.Size([1, 32, 256]))


---

## LSTMを用いた言語モデル

LSTMで言語モデルを作ってみよう。前章で作成したモデルのRNN部分をLSTMに変更する。

In [39]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = LSTM(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, hc=None):
        x = self.embedding(x) # (seq_len, embed_size)
        x = self.dropout(x)
        hs, hc = self.lstm(x, hc) # (seq_len, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (seq_len, n_vocab)
        return y, hc

In [ ]:
n_vocab = len(sp)
embed_size = 512
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size, hidden_size).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 10,301,248



---

## 実践

実際にモデルを学習させてみる。

### 学習

ミニバッチ&Truncated BPTT。

In [43]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model, trunc_len=100):
    model.eval()
    ppls = []
    for x, t in test_loader:
        hc = None
        for i in range(0, x.shape[1], trunc_len):
            x_batch = x[:, i:i+trunc_len].to(device)
            t_batch = t[:, i:i+trunc_len].to(device)
            y, hc = model(x_batch, hc)
            loss = loss_fn(y, t_batch)
            ppl = torch.exp(loss).item()
            ppls.append(ppl)
    ppl = sum(ppls) / len(ppls)
    return ppl

def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            hc = None
            for i in range(0, x.shape[1], trunc_len):
                x_batch = x[:, i:i+trunc_len].to(device)
                t_batch = t[:, i:i+trunc_len].to(device)
                optimizer.zero_grad()
                y, (h, c) = model(x_batch, hc)
                loss = loss_fn(y, t_batch)
                loss.backward()
                optimizer.step()
                ppl = torch.exp(loss).item()
                prog.update(ppl, advance=0)
                hc = (h.detach(), c.detach())
            prog.update()

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [44]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

とりあえず20エポック。ぶっちゃけこの学習は先で実装したLSTMが正しく動くことを確認するためのものなので、適当。

In [45]:
train(model, optimizer, trunc_len=100, n_epochs=20, prog_unit=2)

  1-2/20: #################### 100% [00:06:47.99] ppl train: 775.21, test: 442.91 
  3-4/20: #################### 100% [00:06:45.13] ppl train: 352.91, test: 319.36 
  5-6/20: #################### 100% [00:06:46.92] ppl train: 259.37, test: 265.72 
  7-8/20: #################### 100% [00:06:46.21] ppl train: 207.78, test: 231.26 
 9-10/20: #################### 100% [00:06:45.67] ppl train: 174.51, test: 210.65 
11-12/20: #################### 100% [00:06:46.08] ppl train: 152.48, test: 197.49 
13-14/20: #################### 100% [00:06:47.25] ppl train: 134.94, test: 188.42 
15-16/20: #################### 100% [00:06:49.32] ppl train: 121.97, test: 175.96 
17-18/20: #################### 100% [00:06:48.48] ppl train: 111.31, test: 171.46 
19-20/20: #################### 100% [00:06:48.38] ppl train: 102.90, test: 165.25 


20エポック時点のテストデータでのpplが前章のRNNの40エポック時点のものと同等になったので、LSTMによって精度が上がったと見られる。

In [ ]:
model_path = "models/lm_lstm.pth"
torch.save(model.state_dict(), model_path)

### 文章生成

In [46]:
def token_sampling(y):
    y.squeeze_(0)
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = "",
    max_len: int = 100
) -> str:
    model.eval()
    token_ids = sp.encode(start)
    token_ids.insert(0, bos_id)
    x = torch.tensor(token_ids, device=device)
    y, hc = model(x)
    next_token = token_sampling(y)
    token_ids.append(next_token)

    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device)
        y, hc = model(x, hc)
        next_token = token_sampling(y)
        token_ids.append(next_token)

    sentence = sp.decode(token_ids)
    return sentence

In [47]:
for _ in range(5):
    print(generate_sentence(model, max_len=100))

一般政府の高重な改造により、基本的には死風・革命・ネタなどにより抗技術を用いる事があり、「いっとは島方であったよりも規則より大禁を与えるプーズは、地謀権ないため戦争をもたらしており合画に利用される措置が決わぜられる」と自雑感謝といったものに普及した。この植物の一つである5%が36700円であった。14.108計画は、年間ロー国選挙区
1977年女子府独島府の当時フグチュコッピ郡と南の橋市民の間に生まれ、相作中から中央方へ向かったことから、新たに干名をくどっていた首都との兄弟は、戦後からの足運地を奪い、ブラゼルシアにあった魚雷を創つとして中国名的乗りきをさせたユタブリー・アーボンの亡きで、これを40時代の起航したこともあったものの、度才的には増え
1899年ごろに修行制度を比較させ、超微の野官と共に行われば、国内語で実施されている。このように、パーツが回復し、当時動きに立ち立てられた新取引(壱員の団体への基礎)が樹置している。
性差別安定研究として膵しくれるが、技術組織は減少を受けたシャレバーションの次に適ざ工業機構を設置してソフトアナを設置した。こうした作用能力と採用され、その報道が欧州ワブという環境密事患者は法人税変更に当たってしまう。人件の低解機として提唱された甲生に対するソログとしては、連邦空軍内部武装の3例での設計法を維持することが懸覚的証明のみで
鼓人の嵩地により1963年に破壊されたピッタット26つ貴範。この熱速度に関しては1基がサロウ態台である。公園構造や滝の誉炭神経観測、メキシコに摩擦配陸体を使う鉱害器が広ぜ、帝国観測には適地式で販売できない高速種の差容度のため造う-艦の水防の高い共通酵を有する男を江戸のように残列する1998年の材料


### モデルの改良

例によって私が実装したものは効率が悪い様なのでPyTorchのLSTMを使う。また、LSTM層を増やしてみる。

In [48]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(
            embed_size,
            hidden_size,
            batch_first=True,
            num_layers=3, # 3層にする
            dropout=0.2, # dropoutを指定
        )
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, hc=None):
        x = self.embedding(x) # (seq_len, embed_size)
        x = self.dropout(x)
        hs, hc = self.lstm(x, hc) # (seq_len, hidden_size)
        y = self.fc(hs) # (seq_len, n_vocab)
        return y, hc

`num_layers=3`を指定した。これは以下のモデルと同義。

```python
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, hc=None):
        h, c = hc
        h1, h2, h3 = h
        c1, c2, c3 = c
        x = self.embedding(x) # (seq_len, embed_size)
        x = self.dropout(x)
        hs, (h1, c1) = self.lstm1(x, hc) # (seq_len, hidden_size)
        hs = self.dropout(hs)
        hs, (h2, c2) = self.lstm2(hs, (h1, c1)) # (seq_len, hidden_size)
        hs = self.dropout(hs)
        hs, (h3, c3) = self.lstm3(hs, (h2, c2)) # (seq_len, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (seq_len, n_vocab)
        h = torch.stack([h1, h2, h3]) # (3, batch_size, hidden_size)
        c = torch.stack([c1, c2, c3]) # (3, batch_size, hidden_size)
        hc = (h, c)
        return y, hc
```

In [55]:
n_vocab = len(sp)
embed_size = 512
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size, hidden_size).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 14,503,744


LSTM層を増やしたのでパラメータ数も増えた。これで学習させる。

In [50]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [51]:
train(model, optimizer, trunc_len=100, n_epochs=40, prog_unit=2)

  1-2/40: #################### 100% [00:06:46.66] ppl train: 745.52, test: 410.41 
  3-4/40: #################### 100% [00:06:45.84] ppl train: 321.31, test: 291.90 
  5-6/40: #################### 100% [00:06:47.76] ppl train: 234.03, test: 241.56 
  7-8/40: #################### 100% [00:06:47.57] ppl train: 188.21, test: 213.68 
 9-10/40: #################### 100% [00:06:44.54] ppl train: 158.58, test: 194.58 
11-12/40: #################### 100% [00:06:46.81] ppl train: 138.47, test: 182.26 
13-14/40: #################### 100% [00:06:45.40] ppl train: 123.29, test: 174.50 
15-16/40: #################### 100% [00:06:42.63] ppl train: 111.80, test: 167.09 
17-18/40: #################### 100% [00:06:49.01] ppl train: 101.88, test: 161.07 
19-20/40: #################### 100% [00:06:46.16] ppl train: 94.54, test: 158.95 
21-22/40: #################### 100% [00:06:45.63] ppl train: 88.44, test: 155.21 
23-24/40: #################### 100% [00:06:51.86] ppl train: 82.90, test: 152.29 
25-26/4

20エポック時点までのpplの減りは少し良くなったが、以降は減りが停滞してしまった。

In [52]:
model_path = "models/lm_lstm_imp.pth"
torch.save(model.state_dict(), model_path)

In [53]:
for _ in range(5):
    print(generate_sentence(model, max_len=100))

戦闘船団は、メキシコ西部618Nmであり、大多数が3000型2をビリウム・カングナ(シノン・ラン)のもの規模で1535cm39で走行するという方式で、バラエティに大きく大きな足望を示した。モンドと当時の自動車では、古いガス座・綿製の鋳橋が捕らえた高高度な特許を持っていた場所であり、家庭に形式され、これら奴隷流通区が賃貸している。2156
添血行動の資料は図形である。周囲の大きい宇宙通信機の天元は早くもまるでに、とりわけ虚なる市民(そして表想士)に加え、この地球で一ノく地地点と非常に白い防衛を行うことがないため、変化が若干伝わったならば、餌を濁すので強面をかけた。幼虫を防ぐほど、乾燥した私と、ますます低を超える微微性を編
G unoS_oughは、地球内でIPChadil Veadic Dattoemch Grak's’onen Pulder 自動の進化したものでしかない。ちなみに、主に、いずれもUhaavodBという「ホワイト・ウォーカー」(Mul物 haviioal Dove )という用語を執筆しながら独立した技術な用語を実現させることができる。
清常顧問武装・予備の水戸蓄太 戸談同然。初めて有名な商科一行と、間接的な痛光種回りに者計雌達、避白種の男女でデザイン的に「読者が教界者とするべき烈丈な事柄減りの公理を使って使えない」具体的な体験を受けた。腫瘍では精神保健が必要を実施しており、実際には「使用機」という、社会自殺、110%、13%について
出校者(一部)は同校36号部校9計2号 近く下 区附区分郡舘2, 2013年11月13日 〈山口紗梨駅分東京 - 体丸林山 - 海軍場佐敷)。
